#### Naive Bayes Model

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

#### Load dataset and shape of the dataset

In [39]:
data_frame = pd.read_csv("FinalData.csv")

data_frame.shape


(309226, 10)

#### Dataset values, head(10)

In [40]:
data_frame.head(10)

,VAX_TYPE,AGE_YRS,SEX,SLEEPING_PROB,CHEST_TIGHTNESS,BREATH,COUGH,ALLERGY,WHEEZING,ASTHMA
0,HEPA,1.50,M,False,False,False,False,False,False,False
1,RV5,28.00,F,False,True,True,True,False,True,True
2,FLU3,53.00,F,False,False,False,False,False,False,False
3,HEP,69.00,M,False,False,False,False,False,False,False
4,HEPA,24.00,F,False,False,True,True,False,True,True
5,TDAP,35.00,F,False,False,False,False,False,False,False
6,VARZOS,65.00,F,False,False,False,False,False,False,False
7,PPV,1.58,F,False,False,True,True,False,False,True
8,FLU4,0.33,F,False,False,False,False,False,False,False
9,FLU4,62.00,M,False,False,False,False,False,False,False


#### Data Cleaning. If there any missing values, then delete the corresponding rows for more accuracy

In [41]:
print("Missing values?")
print(data_frame.isnull().values.any())
print("ereasing null values.")
data_frame = data_frame.dropna()

print("Missing values?")
print(data_frame.isnull().values.any())
data_frame.shape

Missing values?
True
ereasing null values.
Missing values?
False


(245860, 10)

#### Data transform: mapping string values with integer values

In [42]:
asthma_map = {True : 1, False : 0}
data_frame['ASTHMA'] = data_frame['ASTHMA'].map(asthma_map)

sleep_map = {True : 1, False : 0}
data_frame['SLEEPING_PROB'] = data_frame['SLEEPING_PROB'].map(sleep_map)

chest_map = {True : 1, False : 0}
data_frame['CHEST_TIGHTNESS'] = data_frame['CHEST_TIGHTNESS'].map(chest_map)

breath_map = {True : 1, False : 0}
data_frame['BREATH'] = data_frame['BREATH'].map(breath_map)

cough_map = {True : 1, False : 0}
data_frame['COUGH'] = data_frame['COUGH'].map(cough_map)

allergy_map = {True : 1, False : 0}
data_frame['ALLERGY'] = data_frame['ALLERGY'].map(allergy_map)

sex_map = {'M' : 1, 'F' : 0, 'U' : 2}
data_frame['SEX'] = data_frame['SEX'].map(sex_map)

wheezing_map = {True : 1, False : 0}
data_frame['WHEEZING'] = data_frame['WHEEZING'].map(wheezing_map)

vax_data = data_frame['VAX_TYPE']
vax=[]
for items in vax_data:
    if items in vax:
        continue
    else:
        #print(items)
        vax.append(items)
#print("Vax list: ")
#print(vax)
key=0
vax_map={key: value for (key, value) in zip(vax,range(len(vax))) }
data_frame['VAX_TYPE'] = data_frame['VAX_TYPE'].map(vax_map)

#### Dataset after mapping

In [43]:
data_frame.head(10)

,VAX_TYPE,AGE_YRS,SEX,SLEEPING_PROB,CHEST_TIGHTNESS,BREATH,COUGH,ALLERGY,WHEEZING,ASTHMA
0,0,1.50,1,0,0,0,0,0,0,0
1,1,28.00,0,0,1,1,1,0,1,1
2,2,53.00,0,0,0,0,0,0,0,0
3,3,69.00,1,0,0,0,0,0,0,0
4,0,24.00,0,0,0,1,1,0,1,1
5,4,35.00,0,0,0,0,0,0,0,0
6,5,65.00,0,0,0,0,0,0,0,0
7,6,1.58,0,0,0,1,1,0,0,1
8,7,0.33,0,0,0,0,0,0,0,0
9,7,62.00,1,0,0,0,0,0,0,0


#### Number of case having asthma or not after vaccination

In [44]:
num_obs = len(data_frame)
num_true = len(data_frame.loc[data_frame['ASTHMA'] == 1])
num_false = len(data_frame.loc[data_frame['ASTHMA'] == 0])
print("Number of True cases:  {0} ({1:2.2f}%)".format(num_true, ((1.00 * num_true)/(1.0 * num_obs)) * 100))
print("Number of False cases: {0} ({1:2.2f}%)".format(num_false, (( 1.0 * num_false)/(1.0 * num_obs)) * 100))

Number of True cases:  16089 (6.54%)
Number of False cases: 229771 (93.46%)


#### Data splitting: 70% train set and 30% test set

In [45]:
from sklearn.model_selection import train_test_split

feature_col_names = ['VAX_TYPE', 'AGE_YRS', 'SEX', 'SLEEPING_PROB', 'CHEST_TIGHTNESS', 'BREATH', 'COUGH', 'ALLERGY','WHEEZING']
predicted_class_names = ['ASTHMA']

X = data_frame[feature_col_names].values     # predictor feature columns (8 X m)
y = data_frame[predicted_class_names].values # predicted class (1=true, 0=false) column (1 X m)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) 

print("split varirfication:")
trainval = (1.0 * len(X_train)) / (1.0 * len(data_frame.index))
testval = (1.0 * len(X_test)) / (1.0 * len(data_frame.index))
print("{0:0.2f}% in training set".format(trainval * 100))
print("{0:0.2f}% in test set".format(testval * 100))

split varirfication:
70.00% in training set
30.00% in test set


#### verifying whether the predicted value was split correctly

In [46]:
print("\nOriginal True  : {0} ({1:0.2f}%)".format(len(data_frame.loc[data_frame['ASTHMA'] == 1]), (len(data_frame.loc[data_frame['ASTHMA'] == 1])/len(data_frame.index)) * 100.0))
print("Original False : {0} ({1:0.2f}%)".format(len(data_frame.loc[data_frame['ASTHMA'] == 0]), (len(data_frame.loc[data_frame['ASTHMA'] == 0])/len(data_frame.index)) * 100.0))
print("")
print("Training True  : {0} ({1:0.2f}%)".format(len(y_train[y_train[:] == 1]), (len(y_train[y_train[:] == 1])/len(y_train) * 100.0)))
print("Training False : {0} ({1:0.2f}%)".format(len(y_train[y_train[:] == 0]), (len(y_train[y_train[:] == 0])/len(y_train) * 100.0)))
print("")
print("Test True      : {0} ({1:0.2f}%)".format(len(y_test[y_test[:] == 1]), (len(y_test[y_test[:] == 1])/len(y_test) * 100.0)))
print("Test False     : {0} ({1:0.2f}%)".format(len(y_test[y_test[:] == 0]), (len(y_test[y_test[:] == 0])/len(y_test) * 100.0)))



Original True  : 16089 (6.54%)
Original False : 229771 (93.46%)

Training True  : 11289 (6.56%)
Training False : 160813 (93.44%)

Test True      : 4800 (6.51%)
Test False     : 68958 (93.49%)


#### Data Scaling

In [47]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test) 

#### Training algorithm: Naive Bayes 

In [48]:
from sklearn.naive_bayes import GaussianNB
# create Gaussian Naive Bayes model object and train it with the data
nb_model = GaussianNB()

nb_model.fit(X_train, y_train.ravel())
# this returns array of predicted results
prediction_from_trained_data = nb_model.predict(X_train)

#### Accuracy on train set and test set

In [49]:
#get accuracy
from sklearn import metrics

accuracy = metrics.accuracy_score(y_train, prediction_from_trained_data)
print("\nAccuracy of NB of train data is : {0:.4f}".format(accuracy))

#print accuracy
# this returns array of predicted results from test_data
prediction_from_test_data = nb_model.predict(X_test)

accuracy = metrics.accuracy_score(y_test, prediction_from_test_data)

print("\nAccuracy of NB on test Data is: {0:0.4f}".format(accuracy))


Accuracy of NB of train data is : 0.9669

Accuracy of NB on test Data is: 0.9671


#### Confusion matrix and classification report

In [50]:
from sklearn.metrics import classification_report, confusion_matrix 

print("Confusion Matrix for NB Network")
print("{0}".format(confusion_matrix(y_test,prediction_from_test_data,labels=[1,0])))  

print("\nClassification Report")
print("{0}".format(classification_report(y_test,prediction_from_test_data, labels=[1, 0]))) 

Confusion Matrix for NB Network
[[ 4800     0]
 [ 2427 66531]]

Classification Report
             precision    recall  f1-score   support

          1       0.66      1.00      0.80      4800
          0       1.00      0.96      0.98     68958

avg / total       0.98      0.97      0.97     73758

